In [2]:
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = 'module_4/part_7'

print(role)
print(sess)
print(region)
print(bucket)
print(prefix)

arn:aws:iam::467432373215:role/service-role/AmazonSageMaker-ExecutionRole-20221206T164397
eu-west-1
sagemaker-eu-west-1-467432373215
module_4/part_7


In [3]:
metric_definitions = [
    {
        "Name": "train:loss",
        "Regex": ".*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*"
    },
    {
        "Name": "train:accuracy",
        "Regex": ".*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*"
    },
    {
        "Name": "validation:accuracy",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*",
    },
    {
        "Name": "validation:loss",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*",
    },
    {
        "Name": "sec/sample",
        "Regex": ".* - \d+s (\d+)[mu]s/sample - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+",
    },
]

In [4]:
import uuid

checkpoint_s3_prefix = f"{prefix}/checkpoints/{str(uuid.uuid4())}"
checkpoint_s3_uri = f"s3://{bucket}/{checkpoint_s3_prefix}/"

print(checkpoint_s3_prefix, checkpoint_s3_uri)

module_4/part_7/checkpoints/eb891094-bffb-4e2c-9f28-f6d0958f21d2 s3://sagemaker-eu-west-1-467432373215/module_4/part_7/checkpoints/eb891094-bffb-4e2c-9f28-f6d0958f21d2/


In [5]:
from sagemaker.tensorflow import TensorFlow

In [6]:
# --------------------------------------------------------------------------------
# 'trainingJobName' msut satisfy regular expression pattern: ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}
# --------------------------------------------------------------------------------
base_job_name = "fashion-mnist-tensorflow"
hyperparameters = {
    "epochs": 2, 
    "batch-size": 64
}

estimator = TensorFlow(
    entry_point="fashion_mnist_tensoflow.py",
    source_dir="tf_code",
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    role=role,
    input_mode='File',
    framework_version="2.3.1",
    py_version="py37",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    base_job_name=base_job_name,
    checkpoint_s3_uri=checkpoint_s3_uri,
    model_dir=False
)
estimator.fit()

2022-12-12 20:14:56 Starting - Starting the training job...
2022-12-12 20:15:19 Starting - Preparing the instances for trainingProfilerReport-1670876095: InProgress
......
2022-12-12 20:16:25 Downloading - Downloading input data
2022-12-12 20:16:25 Training - Downloading the training image......
2022-12-12 20:17:19 Training - Training image download completed. Training in progress.2022-12-12 20:17:16.394704: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-12-12 20:17:16.394861: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-12-12 20:17:16.426376: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-12-12 20:17:17,702 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-12-12 20:17:17,711 

###  Descargando y usando el modelo localmente

In [7]:
estimator.model_data

's3://sagemaker-eu-west-1-467432373215/fashion-mnist-tensorflow-2022-12-12-20-14-55-373/output/model.tar.gz'

In [9]:
!mkdir ./fashion_mnist -p
!aws s3 cp {estimator.model_data} ./fashion_mnist/model.tar.gz
!tar -xzf fashion_mnist/model.tar.gz -C ./fashion_mnist/

download: s3://sagemaker-eu-west-1-467432373215/fashion-mnist-tensorflow-2022-12-12-20-14-55-373/output/model.tar.gz to fashion_mnist/model.tar.gz
tar: fashion_mnist/model: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [19]:
import tensorflow as tf
my_model=tf.keras.models.load_model('./fashion_mnist/00000000')

In [20]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
